# Restoring Old Data
The goal of this notebook is to restore all old data (from a local copy) to the database for historical analysis of previous models. 

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import os
import pymssql

In [3]:
from os import getenv

connection = pymssql.connect(
    host=getenv("SQL_HOST"),
    port=getenv("SQL_PORT"),
    user=getenv("SQL_USER"),
    password=getenv("SQL_PASSWORD"),
    database=getenv("SQL_DATABASE"),
)

In [4]:
def load_historical_data(name, epochs):
  '''
  Epochs must be in the form:
  [
    {
      'epoch': epoch,
      'loss': loss,
      'reward': reward
    }
  ]
  '''
  cursor = connection.cursor(as_dict=True)
  cursor.execute(f'INSERT INTO Models (ModelName) OUTPUT Inserted.Modelid VALUES (\'Historical: {name}\')')
  row = cursor.fetchone()
  id = row['Modelid']

  max_len = 500

  for rows in [epochs[i * max_len: i * max_len + max_len] for i in range((len(epochs) // max_len) + 1)]:
    if len(rows) > 0:
      cursor.execute(f"INSERT INTO Epochs (Modelid, Epoch, Loss, Reward) VALUES " + ",".join([
        f'({id}, {row["epoch"]}, {row["loss"]}, {row["reward"]})'
        for row in rows
      ]))


  connection.commit()
  cursor.close()

In [5]:
def load_json(file):
  with open(file) as f:
    return json.load(f)

In [6]:
base_path = '/mnt/f/Github/mlcube_archive/mlcube/agents'

In [7]:
for name in ['1','2','3','4','5','6','7','8','9','10','11','12','13','15','16','A-1','A-2','A-3','agent-1','agent-2','B-1','B-2','B-3','B-4','B-5','main-1','main-2','main-3','main-4','main-5','main-6','relu-1','relu-2','trial-1']:
  print(f'Loading {name}')
  data = load_json(f'{base_path}/{name}/epochs.json')
  load_historical_data(name, data)


Loading 1
Loading 2
Loading 3
Loading 4
Loading 5
Loading 6
Loading 7
Loading 8
Loading 9
Loading 10
Loading 11
Loading 12
Loading 13
Loading 15
Loading 16
Loading A-1
Loading A-2
Loading A-3
Loading agent-1
Loading agent-2
Loading B-1
Loading B-2
Loading B-3
Loading B-4
Loading B-5
Loading main-1
Loading main-2
Loading main-3
Loading main-4
Loading main-5
Loading main-6
Loading relu-1
Loading relu-2
Loading trial-1
